# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [44]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2data as bd
import bw2calc as bc
import bw_graph_tools as graph
import numpy as np
import pandas as pd

In [45]:
bd.projects.set_current('default')
bd.projects

Brightway2 projects manager with 15 objects:
	LCA_VL_masterdb
	Test
	as23_project
	as23_project_bw2
	default
	ei39
	ei39_premise_scenarios
	medusa_test1
	premise-example
	premise_ei39
	test
	tictac
	tictacthree
	📊📈💎🕤🗓️
	🪒
Use `projects.report()` to get a report on all projects.

In [46]:
try: 
    bd.projects.delete_project('medusa_test1')
    bd.projects.purge_deleted_directories()
except:
    pass

bd.projects.set_current("medusa_test1")

### Create some example databases
This is based on the forest example

In [47]:
# bd.Database('temporalis-bio').write({
#     ('temporalis-bio', "CO2"): {
#         "type": "emission",
#         "name": "carbon dioxide",
#         "temporalis code": "co2",
#     },
#     ('temporalis-bio', "CH4"): {
#         "type": "emission",
#         "name": "methane",
#         "temporalis code": "ch4",
#     },
# })

# bd.Database('dummy-database').write({
#     ('dummy-database', 'dummy_activity'): {
#         'name': 'Drive an electric car',
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'type': 'production',
#                 'input': ('dummy-database', 'dummy_activity'),
#             },
#             {
#                 'amount': 3,
#                 'type': 'technosphere',
#                 'input': ('dummy-database', 'dummy_activity_2'),
#             },
#         ]
#     },
#     ('dummy-database', 'dummy_activity_2'): {
#         'name': 'Drive an electric car 3',
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'type': 'production',
#                 'input': ('dummy-database', 'dummy_activity_2'),
#             },
#             {
#                 'amount': 398189,
#                 'type': 'biosphere',
#                 'input': ('temporalis-bio', 'CO2'),
#             },
#         ]
#     }
# })


# bd.Database('dummy-database-2').write({
#     ('dummy-database-2', 'dummy_activity'): {
#         'name': 'Drive an electric car',
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'type': 'production',
#                 'input': ('dummy-database-2', 'dummy_activity'),
#             },
#             {
#                 'amount': 1,
#                 'type': 'technosphere',
#                 'input': ('dummy-database-2', 'dummy_activity_2'),
#             },
#         ]
#     },
#     ('dummy-database-2', 'dummy_activity_2'): {
#         'name': 'Drive an electric car 3',
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'type': 'production',
#                 'input': ('dummy-database-2', 'dummy_activity_2'),
#             },
#             {
#                 'amount': 0,
#                 'type': 'biosphere',
#                 'input': ('temporalis-bio', 'CO2'),
#             },
#         ]
#     }
# })


# bd.Database('temporalis-example').write({
#     ('temporalis-example', 'Functional Unit'): {
#         'name': 'Functional Unit',
#         'type': 'process',
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Functional Unit'),
#                 'type': 'production',
#             },
#             {
#                 'amount': 5,
#                 'input': ('temporalis-example', 'EOL'),
#                 'temporal_distribution': 
#                     easy_timedelta_distribution(
#                     start=0,
#                     end=4, # Range includes both start and end
#                     resolution="Y",  # M for months, Y for years, etc.
#                     steps=5,
#                 ),
#                 'type': 'technosphere'
#             },
#         ],
#     },
#     ('temporalis-example', 'EOL'): {
#         'exchanges': [
#             {
#                 'amount': 0.8,
#                 'input': ('temporalis-example', 'Waste'),
#                 'type': 'technosphere'
#             },
#             {
#                 'amount': 0.2,
#                 'input': ('temporalis-example', 'Landfill'),
#                 'type': 'technosphere'
#             },
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Use'),
#                 'type': 'technosphere'
#             },
#         ],
#         'name': 'EOL',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Use'): {
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Production'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([-4], dtype='timedelta64[M]'),
#                     np.array([1.0])
#                 ),
#                 'type': 'technosphere'
#             },
#         ],
#         'name': 'Use',
#     },
#     ('temporalis-example', 'Production'): {
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Transport'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([200],dtype='timedelta64[D]'),
#                     np.array([1.0])
#                 ),
#                 'type': 'technosphere'
#             },
#         ],
#         'name': 'Production',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Transport'): {
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Sawmill'),
#                 'type': 'technosphere'
#             },
#             {
#                 'amount': 0.1,
#                 'input': ('temporalis-bio', 'CO2'),
#                 'type': 'biosphere'
#             },
#         ],
#         'name': 'Production',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Sawmill'): {
#         'exchanges': [
#             {
#                 'amount': 1.2,
#                 'input': ('temporalis-example', 'Forest'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([-14], dtype='timedelta64[M]'),
#                     np.array([1.0])
#                 ),
#                 'type': 'technosphere'
#             },
#             {
#                 'amount': 0.1,
#                 'input': ('temporalis-bio', 'CO2'),
#                 'type': 'biosphere'
#             },
#             {
#                 'amount': 4,
#                 'input': ('dummy-database', 'dummy_activity'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([-5], dtype='timedelta64[M]'),
#                     np.array([1.0])
#                 ),
#                 'type': 'technosphere'
#             },
#         ],
#         'name': 'Sawmill',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Forest'): {
#         'exchanges': [
#             {
#                 'amount': -.1 * 6,
#                 'input': ('temporalis-bio', 'CO2'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([-4, -3, 0, 1, 2, 5], dtype='timedelta64[Y]'),
#                     np.ones(6) * (1/6)
#                 ),
#                 'type': 'biosphere'
#             },
#             {
#                 'amount': 1.5,
#                 'input': ('temporalis-example', 'Thinning'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([-3, 0, 1], dtype='timedelta64[Y]'),
#                     np.ones(3) * 1/3
#                 ),
#                 'type': 'technosphere'
#             },
#         ],
#         'name': 'Forest',
#     },
#     ('temporalis-example', 'Thinning'): {
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Thinning'),
#                 'type': 'production'
#             },
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Avoided impact - thinnings'),
#                 'type': 'production'
#             },
#         ],
#         'name': 'Thinning',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Landfill'): {
#         'exchanges': [
#             {
#                 'amount': 0.1,
#                 'input': ('temporalis-bio', 'CH4'),
#                 'temporal_distribution': TemporalDistribution(
#                     np.array([10, 20, 60, 100], dtype='timedelta64[M]'),
#                     np.ones(4) * 1/4
#                 ),
#                 'type': 'biosphere'
#             },
#         ],
#         'name': 'Landfill',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Waste'): {
#         'exchanges': [
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Waste'),
#                 'type': 'production'
#             },
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Avoided impact - waste'),
#                 'type': 'production'
#             },
#         ],
#         'name': 'Waste',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Avoided impact - waste'): {
#         'exchanges': [
#             {
#                 'amount': -0.4,
#                 'input': ('temporalis-bio', 'CO2'),
#                 'type': 'biosphere'
#             },
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Avoided impact - waste'),
#                 'type': 'production'
#             },
#         ],
#         'name': 'Avoided impact - waste',
#         'type': 'process'
#     },
#     ('temporalis-example', 'Avoided impact - thinnings'): {
#         'exchanges': [
#             {
#                 'amount': -0.3,
#                 'input': ('temporalis-bio', 'CO2'),
#                 'type': 'biosphere'
#             },
#             {
#                 'amount': 1,
#                 'input': ('temporalis-example', 'Avoided impact - thinnings'),
#                 'type': 'production'
#             },
#         ],
#         'name': 'Avoided impact - thinnings',
#         'type': 'process'
#     }
# })


In [48]:
bd.Database('temporalis-bio').write({
    ('temporalis-bio', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-bio', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
})

bd.Database('background_2023').write({
    ('background_2023', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_wind'),
            },
        ]
    },
    ('background_2023', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2023', 'electricity_wind'),
            },
            {
                'amount': 1,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('background_2020').write({
    ('background_2020', 'electricity_mix'): {
        'name': 'Electricity mix',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_mix'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2020', 'electricity_wind'),
            },
        ]
    },
    ('background_2020', 'electricity_wind'): {
        'name': 'Electricity production, wind',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('background_2020', 'electricity_wind'),
            },
            {
                'amount': 3124987,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})

bd.Database('foreground').write({
    ('foreground', 'heat_from_hydrogen'): {
        'name': 'Heat production, hydrogen',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'heat_from_hydrogen'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('foreground', 'electrolysis'),
                'temporal_distribution': # e.g. because some hydrogen was stored in the meantime
                    easy_timedelta_distribution(
                    start=-4,
                    end=0, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=2,
                ),
            },
        ]
    },('foreground', 'electrolysis'): {
        'name': 'Hydrogen production, electrolysis',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('foreground', 'electrolysis'),
            },
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('background_2023', 'electricity_mix'),
            },
        ]
    },
})

100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]

Vacuuming database 


In [49]:
bd.Method(("GWP", "example")).write([
    (('temporalis-bio', "CO2"), 1),
    (('temporalis-bio', "CH4"), 25),
])

In [50]:
demand = {('foreground', 'heat_from_hydrogen'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [51]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
slca.score

1.0

In [52]:
# [(bd.get_node(id=i)['name'], bd.get_node(id=i)['database']) for i in lca.dicts.product.reversed.values()]

# Custom `EdgeExtractor` class

In [53]:
from dataclasses import dataclass
from heapq import heappop, heappush
from typing import Callable, List

from bw_temporalis import TemporalisLCA, TemporalDistribution


@dataclass
class Edge:
    """
    Class for storing a temporal edge with source and target.

    Leaf edges link to a source process which is a leaf in
    our graph traversal (either through cutoff or a filter
    function).

    Attributes
    ----------
    distribution : TemporalDistribution
    leaf : bool
    consumer : int
    producer : int

    """

    distribution: TemporalDistribution
    leaf: bool
    consumer: int
    producer: int


class EdgeExtracter(TemporalisLCA):
    def __init__(self, *args, edge_filter_function: Callable = None, **kwargs):
        super().__init__(*args, **kwargs)
        if edge_filter_function:
            self.edge_ff = edge_filter_function
        else:
            self.edge_ff = lambda x: False

    def build_edge_timeline(self, node_timeline: bool | None = False) -> List:
        heap = []
        timeline = []

        for edge in self.edge_mapping[self.unique_id]:
            node = self.nodes[edge.producer_unique_id]
            heappush(
                heap,
                (
                    1 / node.cumulative_score,
                    self.t0 * edge.amount,
                    node,
                ),
            )
            timeline.append(
                Edge(
                    distribution=self.t0 * edge.amount,
                    leaf=False,
                    consumer=self.unique_id,
                    producer=node.activity_datapackage_id,
                )
            )

        while heap:
            _, td, node = heappop(heap)

            for edge in self.edge_mapping[node.unique_id]:
                row_id = self.nodes[edge.producer_unique_id].activity_datapackage_id
                col_id = node.activity_datapackage_id
                exchange = self.get_technosphere_exchange(
                    input_id=row_id,
                    output_id=col_id,
                )
                value = (
                    self._exchange_value(
                        exchange=exchange,
                        row_id=row_id,
                        col_id=col_id,
                        matrix_label="technosphere_matrix",
                    )
                    / node.reference_product_production_amount
                )
                producer = self.nodes[edge.producer_unique_id]
                leaf = self.edge_ff(row_id)

                distribution = (td * value).simplify()
                timeline.append(
                    Edge(
                        distribution=distribution,
                        leaf=leaf,
                        consumer=node.activity_datapackage_id,
                        producer=producer.activity_datapackage_id,
                    )
                )
                if not leaf:
                    heappush(
                        heap,
                        (
                            1 / node.cumulative_score,
                            distribution,
                            producer,
                        ),
                    )
        return timeline

In [54]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2023')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

# Example

In [55]:
eelca = EdgeExtracter(slca, edge_filter_function=filter_function)

Starting graph traversal
Calculation count: 3


In [56]:
timeline = eelca.build_edge_timeline()

In [57]:
[edge.distribution.date for edge in timeline]

[array(['2023-10-14T11:38:32'], dtype='datetime64[s]'),
 array(['2019-10-14T12:21:44', '2023-10-14T11:38:32'],
       dtype='datetime64[s]'),
 array(['2019-10-14T12:21:44', '2023-10-14T11:38:32'],
       dtype='datetime64[s]')]

In [58]:
from medusa_tools import create_grouped_edge_dataframe
from datetime import datetime

dates_list = [
        datetime.strptime("2020", "%Y"),
        datetime.strptime("2023", "%Y"),
    ] # we probably should comebine this with database_date_dict, or include the time data in the databases directly
timeline_df = create_grouped_edge_dataframe(timeline, dates_list, interpolation_type="linear")

c:\Users\timod\OneDrive\Dokumente\Python\tictac_lca\medusa_tools.py:334: Warning: Date outside the range of dates covered by the databases.
  warnings.warn("Date outside the range of dates covered by the databases.", category=Warning)


In [59]:
timeline_df
# if leaf -> include prospective db in dp???

,year,producer,consumer,amount,leaf,date,interpolation_weights,producer_name
0,2019,11,16,0.5,1,2019-01-01,{2020: 1},Electricity mix
1,2019,16,15,0.5,0,2019-01-01,{2020: 1},"Hydrogen production, electrolysis"
2,2023,11,16,0.5,1,2023-01-01,{2023: 1},Electricity mix
3,2023,15,-1,1.0,0,2023-01-01,{2023: 1},"Heat production, hydrogen"
4,2023,16,15,0.5,0,2023-01-01,{2023: 1},"Hydrogen production, electrolysis"


In [60]:
from medusa_tools import get_datapackage_from_edge_timeline

database_date_dict = {
            2020: 'background_2020',
            2023: 'background_2023',
        }
dp = get_datapackage_from_edge_timeline(timeline_df, database_date_dict)

Choosing background_2020 for year 2020; Previous producer was ('background_2023', 'electricity_mix')
Previous producer: ('background_2023', 'electricity_mix'), id = 11
Previous consumer: ('foreground', 'electrolysis'), id = 16
New producer id = 13
New consumer id = 1602019

Choosing background_2023 for year 2023; Previous producer was ('background_2023', 'electricity_mix')
Previous producer: ('background_2023', 'electricity_mix'), id = 11
Previous consumer: ('foreground', 'electrolysis'), id = 16
New producer id = 11
New consumer id = 1602023



c:\Users\timod\OneDrive\Dokumente\Python\tictac_lca\medusa_tools.py:169: Warning: 
May I have your attention please? Will the real producer please stand up? 
'Heat production, hydrogen' (None, None, None) has no producer. Production exchange missing?
  warnings.warn(f"\nMay I have your attention please? Will the real producer please stand up? \n{bd.get_node(id=row.producer)} has no producer. Production exchange missing?", category=Warning)


In [61]:
# databases_as_dp = [bd.Database(db).datapackage() for db in bd.databases]
# should rather be something like database_names = set.union(
        #     *[
        #         Database(db_label).find_graph_dependents()
        #         for db_label in demand_database_names
        #     ]
        # )

# databases_as_dp = bd.Database('background_2020').datapackage()

In [62]:
from medusa_tools import prepare_medusa_lca_inputs

# fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=gwp)
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, method=gwp) # using this instead of bd.prepare_lca_inputs(), all databases (incl. prospective ones) should now be included in data_objs

In [68]:
remapping

{'activity': {9: ('temporalis-bio', 'CO2'),
  10: ('temporalis-bio', 'CH4'),
  11: ('background_2023', 'electricity_mix'),
  12: ('background_2023', 'electricity_wind'),
  13: ('background_2020', 'electricity_mix'),
  14: ('background_2020', 'electricity_wind'),
  15: ('foreground', 'heat_from_hydrogen'),
  16: ('foreground', 'electrolysis')},
 'product': {9: ('temporalis-bio', 'CO2'),
  10: ('temporalis-bio', 'CH4'),
  11: ('background_2023', 'electricity_mix'),
  12: ('background_2023', 'electricity_wind'),
  13: ('background_2020', 'electricity_mix'),
  14: ('background_2020', 'electricity_wind'),
  15: ('foreground', 'heat_from_hydrogen'),
  16: ('foreground', 'electrolysis')},
 'biosphere': {9: ('temporalis-bio', 'CO2'),
  10: ('temporalis-bio', 'CH4'),
  11: ('background_2023', 'electricity_mix'),
  12: ('background_2023', 'electricity_wind'),
  13: ('background_2020', 'electricity_mix'),
  14: ('background_2020', 'electricity_wind'),
  15: ('foreground', 'heat_from_hydrogen'),
 

In [71]:
data_objs[0]['name']

TypeError: 'Datapackage' object is not subscriptable

In [65]:
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)

In [66]:
lca.load_lci_data(nonsquare_ok=True)
acts = set([v for v in lca.dicts.activity.reversed.values()])
prods = set([v for v in lca.dicts.product.reversed.values()])
prods-acts
[bd.get_node(id=missing_product).key for missing_product in prods-acts]

[]

In [67]:
lca.lci()
lca.lcia()
print('new lca score:', lca.score)
print('old lca score:', slca.score)

new lca score: 1.0
old lca score: 1.0


In [43]:
acts

{3, 4, 5, 6, 7, 8, 302019, 302023, 702019, 702023, 802019, 802023}